In [1]:
import os
from langchain_chroma import Chroma

In [2]:
import sys
print(sys.executable)

/Users/zsk4gm/Desktop/resilience_education/env/bin/python3


## Initiate Chroma DB with saved embeddings

In [3]:
# Create the file path to the 'embeddings' folder
current_directory = os.getcwd()
persist_directory = os.path.join(current_directory, 'embeddings')

In [4]:
# Load Embedding Model 
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "multi-qa-mpnet-base-dot-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Create the vector store and specify the persist directory\n",
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=hf)

## Load LLM

In [6]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

class CustomOllama(Ollama):
    def __init__(self, model, callback_manager=None, stop=None, temperature=0.2):
        super().__init__(model=model, stop=stop, temperature=temperature)
        self.callback_manager = callback_manager

# Initialize the CallbackManager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Create an instance of the CustomOllama class
llm = CustomOllama(
    model="llama3",
    callback_manager=callback_manager
)

In [19]:
#test 
prompt = ["Why is the sky blue?"]  # Prompt should be a list of strings

# Generate text using the Ollama model
generated_text = llm.generate(prompts=prompt)


KeyboardInterrupt: 

In [8]:
response = generated_text.generations[0][0].text
print(response)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's what happens:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Molecules**: The atmosphere is made up of tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Scattering**: When sunlight encounters these molecules, it scatters in all directions. This scattering effect is more pronounced for shorter wavelengths (like blue and violet) than longer wavelengths (like red and orange).
4. **Blue dominance**: As a result of this scattering, the shorter wavelengths of light (blue and violet) are distributed throughout the atmosphere, reaching our eyes from all parts of the sky.
5. **Red light prevails**: Meanwhile, the longer wavelengths of light (red and orange) continue to 

## Setup RAG

### Instantiate retriever

In [9]:
plain_retriever = vectorstore.as_retriever()

Meeting Notes 7/3/24
Sophia wants to add levels of certainty ('we do not know the answer and you might need to do my research/need to provide more information')
USE felony or misdemeanor instead of crime.
To check accuracy of model try making small changes to prompts and see if it gives us different answers. 
Maybe run 2 prompts as the same time e.g. 'what can I do and what am I not allowed to do'

### Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    '''
    You are a compassionate legal expert tasked with translating Virginia legal restrictions into helpful plaintext for jobseekers with felony or misdemeanor convictions. Your goal is to help them understand what jobs or certifications they can pursue while maintaining a supportive and encouraging tone.

1. Begin with a brief disclaimer: Remind the user that you cannot provide personalized legal advice and that all information is general. Emphasize the importance of consulting with a legal professional for specific guidance.

2. If not provided, ask for relevant details about the user's specific situation (e.g., type of conviction, how long ago it occurred) to provide more accurate information.

3. Use only the following sections of the Virginia legal code to answer the user's query:
   {context}

4. Provide a clear and concise answer addressing the user's query, including as many relevant details as possible from the context. Always cite the specific section of the code you're referencing.

5. If there are restrictions that employers can waive, describe those options clearly.

6. If you are uncertain about any of the restrictions or if none of the sections of the code answer the query, state your uncertainty and recommend consulting a legal professional.

7. Suggest similar jobs or certifications that the user can legally pursue with their conviction in Virginia. Provide a brief explanation of the professional requirments for each suggestion.

8. Encourage the user to conduct further research and provide suggestions for additional resources they can consult (e.g., state employment agencies, legal aid organizations).

9. Conclude with a supportive message, reminding the user that there are often pathways to employment despite past convictions.

Remember to maintain a balance between providing accurate information and offering encouragement to the jobseeker.

Example Query: What jobs can I get in healthcare if I've been convicted of a drug offense?
Example Response: 

    '''
)

prompt_plain = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
prompt_plain.pretty_print()

================================ System Message ================================


    You are a compassionate legal expert tasked with translating Virginia legal restrictions into helpful plaintext for jobseekers with felony or misdemeanor convictions. Your goal is to help them understand what jobs or certifications they can pursue while maintaining a supportive and encouraging tone.

1. Begin with a brief disclaimer: Remind the user that you cannot provide personalized legal advice and that all information is general. Emphasize the importance of consulting with a legal professional for specific guidance.

2. If not provided, ask for relevant details about the user's specific situation (e.g., type of conviction, how long ago it occurred) to provide more accurate information.

3. Use only the following sections of the Virginia legal code to answer the user's query:
   {context}

4. Provide a clear and concise answer addressing the user's query, including as many relevant details as pos

### Chain

Adding memory to the chain - https://python.langchain.com/v0.1/docs/modules/memory/adding_memory/

In [11]:
from typing import List, Dict
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap

# Function to format documents with their metadata
def format_docs(docs: List[Document]):
    formatted_docs = []
    for doc in docs:
        metadata_str = "\n".join(f"{key}: {value}" for key, value in doc.metadata.items())
        formatted_doc = f"Metadata:\n{metadata_str}\n\nContent:\n{doc.page_content}"
        formatted_docs.append(formatted_doc)
    return "\n\n".join(formatted_docs)

# Chain to format documents, process with LLM, and parse the output
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_plain  # Assuming 'prompt' is defined somewhere in your setup
    | llm  # Assuming 'llm' is defined somewhere in your setup
    | StrOutputParser()
)

# Function to retrieve documents based on a query
retrieve_docs = (lambda x: x["input"]) | plain_retriever  # Assuming 'retriever' is defined somewhere in your setup

# Define the final chain
chain_plain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

### Test with prompt

In [12]:
# Assuming you have a string for your prompt template
query = "Can I be a teacher with a violent crime on my record?"

In [15]:
import time 
start_time = time.time()

# Execute the function
result = chain_plain.invoke({"input": query})

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

In [16]:
print(elapsed_time)

28.823548793792725


In [17]:
print(result['answer'])

I'm happy to help you understand your options as a teacher in Virginia despite having a conviction for a violent crime.

According to § 22.1-296.1, the fact of such finding, after all rights to any administrative appeal provided by § 63.2-1526 have been exhausted, shall be grounds for the local school division to recommend that the Board of Education revoke such person's license to teach (SectionTitle: Data on convictions for certain crimes and child abuse and neglect required; penalty).

However, it's important to note that no teacher shall be dismissed or placed on probation solely on the basis of the teacher's refusal to submit to a polygraph examination requested by the school (Terms of Employment Generally, Article 2, Chapter 15).

If you're concerned about your eligibility to teach in Virginia, I recommend consulting with a legal professional who can provide personalized guidance.

In terms of alternative career paths, you may consider pursuing certifications or jobs that don't r

In [16]:
result['context']

[Document(metadata={'ArticleName': 'Terms of Employment Generally', 'ArticleNum': '2', 'ChapterName': 'Teachers, Officers and Employees', 'ChapterNum': '15', 'Hrefs': '/vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; http://lis.virginia.gov/cgi-bin/legp604.exe?961+ful+CHAP0960; http://lis.virginia.gov/cgi-bin/legp604.exe?971+ful+CHAP0103; http://lis.virginia.gov/cgi-bin/legp604.exe?031+ful+CHAP0723; http://lis.virginia.gov/cgi-bin/legp604.exe?061+ful+CHAP0790; http://lis.virginia.gov/cgi-bin/legp604.exe?071+ful+CHAP0245; http://lis.virginia.gov/cgi-bin/legp604.exe?071+ful+CHAP0431; http://lis.virginia.gov/cgi-bin/legp604.exe?081+ful+CHAP0555; http://lis.virginia.gov/cgi-bin/legp604.exe?201+ful+CHAP0877; http://lis.virginia.gov/cgi-bin/legp604.exe?231+ful+CHAP0670; http://lis.virginia.gov/cgi-bin/legp604.exe?231+ful+CHAP0703', 'SectionNumber': '22.1-296.1', 'SectionTitle': 'Data on convictions for certain

In [17]:
query2 = "What jobs can I get in healthcare if I've been convicted of a drug offense?"
result2 = chain_plain.invoke({"input": query2})

In [18]:
print(result2['answer'])

I'm happy to help you explore healthcare job opportunities in Virginia despite your conviction. According to Virginia Code § 19.2-392.02, national criminal background checks are required for businesses and organizations that provide care to children or the elderly or disabled.

While some healthcare jobs may be restricted due to your conviction, there are still many options available. Here are a few suggestions:

1. **Medical Records Technician**: With a conviction, you might face restrictions in certain roles, but medical records technicians work with patient data and don't require direct patient care.
2. **Healthcare Administrator**: Your experience can be valuable in administrative roles, such as managing healthcare facilities or programs.
3. **Pharmacy Technician**: This role involves assisting pharmacists with medication management and doesn't require direct patient contact.

Keep in mind that some healthcare jobs may still be available to you, but it's essential to research the s

In [21]:
query2a = "What jobs can I get in healthcare if I've been convicted of a drug offense and don't require a waiver?"
result2a = chain_plain.invoke({"input": query2a})
print(result2a['answer'])

I'm happy to help you with that!

According to Virginia Code § 37.2-314, the Department may hire individuals for compensated employment at adult substance abuse or mental health treatment programs who were previously convicted of certain drug-related offenses, as long as those convictions occurred more than four years prior to the application date.

In your case, since you've been convicted of a drug offense and don't require a waiver, you may be eligible for jobs in healthcare that involve working at adult substance abuse or mental health treatment programs. Some examples of jobs you might consider include:

1. Case Manager: You could work with clients who are struggling with addiction, helping them develop plans to overcome their dependencies.
2. Counselor: With training and certification, you could provide counseling services to individuals dealing with substance abuse issues.
3. Support Specialist: You could assist patients in treatment programs, providing emotional support and hel

In [22]:
query2b = "What certifications or trainings could I get"
result2b = chain_plain.invoke({"input": query2b})
print(result2b['answer'])

I'm happy to help you explore certification and training options in Virginia. According to Section 54.1-3446, Schedule I of the Professions and Occupations title, there are certain restrictions on professions and occupations that may be affected by your conviction.

However, it's essential to note that not all certifications or trainings require a specific profession or occupation. Here are some alternative options you might consider:

1. **Certified Nursing Assistant (CNA)**: You can pursue a CNA certification, which typically requires a training program and passing an exam. This certification is not directly related to your conviction.
2. **Food Safety Certification**: The Virginia Department of Agriculture and Consumer Services offers food safety certifications for individuals working in the food industry. This certification is not restricted by your conviction.
3. **Basic Life Support (BLS) Certification**: You can obtain a BLS certification, which is essential for many healthcare 

In [20]:
query3 = "What is Taylor Swift's latest album called?"
result3 = chain_plain.invoke({"input": query3})
print(result3['answer'])

I'm happy to help you with your question! However, I need to clarify that this conversation is about Virginia legal restrictions, not music. As a compassionate legal expert, my goal is to provide you with accurate information and helpful resources.

To answer your original query, since it's not related to the provided code section (55.1-504), I'd be happy to help you explore other topics or questions. Please feel free to ask me anything about Virginia legal restrictions, and I'll do my best to assist you.

If you're interested in learning more about job opportunities despite past convictions in Virginia, I can provide some suggestions. For instance, you may want to consider certifications like the Certified Nursing Assistant (CNA) or the Certified Medical Administrative Assistant (CMA). These certifications can be valuable assets when applying for jobs in healthcare settings.

Additionally, you can consult with state employment agencies, such as the Virginia Employment Commission, or l

# Evaluate RAG